In [1]:
import numpy as np
import pandas as pd
import json
from importlib import reload 
import statsmodels.api as sm
import bright_future_base as bf
reload(bf)

<module 'bright_future_base' from '/home/shinjini/Shinjini/MIT/Courses/Spring 2022/6.S079/Project/Code/bright-future/src/bright_future_base.py'>

# Example 1 with BrightFuture

In [2]:
# with BrightFuture

df_works_author = bf.load_df("works", grouped_by="author")
df_awards_author = bf.load_df("awards", grouped_by="author")
bf.merge_datasets(["works", "awards"])

,author,adjustedcount,count,award_count
0,,0.000000,0.0,0.0
1,A Min Tjoa,0.533330,2.0,0.0
2,A. Aldo Faisal,2.309520,8.0,0.0
3,A. B. M. Alim Al Islam,0.529759,3.0,0.0
4,A. B. Siddique,0.999990,3.0,0.0
5,A. Cüneyd Tantug,0.666660,2.0,0.0
6,A. David Marshall,1.666650,5.0,0.0
7,A. E. Eiben,0.500000,1.0,0.0
8,A. Ercüment Çiçek,0.666660,2.0,0.0
9,A. Jefferson Offutt,1.833330,5.0,0.0


# Example 1 without BrightFuture

In [3]:
df = pd.read_csv('../data/awards-data.tsv', sep='\t', header=0)

with open('../data/csrankings/authors-small.json') as f:
    authors_small = json.load(f)
df_authors = pd.DataFrame.from_dict(authors_small)
df_authors['count'] = df_authors['count'].astype(float)
df_authors['adjustedcount'] = df_authors['adjustedcount'].astype(float)
df_authors['year'] = df_authors['year'].astype(float)
df_authors = df_authors.rename(columns={"title": "field", "name": "author"})
df_works = df_authors

df_works_author = df_works.groupby("author").sum().reset_index()
df_awards = df.assign(
    award_count=1).groupby("author-name").sum().drop(
    columns=["year"]).reset_index()

merged_df = df_awards.merge(df_works_author,
                            left_on='author-name',
                            right_on='author',
                            how='right')

merged_df = merged_df.fillna(0).drop(columns=["year"])


mod = sm.OLS(merged_df.award_count,
             sm.add_constant(merged_df[["count"]]))
res = mod.fit()
print(res.summary())


mod = sm.OLS(merged_df.award_count,
             sm.add_constant(merged_df[["adjustedcount",
                                        "count"]]))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            award_count   R-squared:                       0.918
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     11.21
Date:                Fri, 13 May 2022   Prob (F-statistic):              0.185
Time:                        00:42:41   Log-Likelihood:                 1.7531
No. Observations:                   3   AIC:                            0.4938
Df Residuals:                       1   BIC:                            -1.309
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2540      0.589      3.825      0.1

/home/shinjini/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/shinjini/.local/lib/python3.6/site-packages/statsmodels/stats/stattools.py:71: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)
/home/shinjini/.local/lib/python3.6/site-packages/statsmodels/stats/stattools.py:71: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)
/home/shinjini/.local/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1698: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/shinjini/.local/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1699: 